---
layout: default
categories: linearRegression
title: "Linear Regression - Implementation"
permalink: /ML4.5/
order: 4.5
comments: true
---

In [ ]:
%pylab --no-import-all inline
plt.rcParams["mathtext.fontset"] = "cm"

# Linear regression implementation
Since linear regression is a trivial model, it is relatively easy to implement it from scratches and maybe in the future I'll implement a full version on this page. 

Many libraries enabling a user to build and train a linear regression model exist. In the last years I feel like `scikit-learn` and `pytorch` are the most widely used libraries in machine learning.

## Reading data
For this example we are using house prices as a function of inhabitable surface and number of rooms. Data is stored in a csv file, to parse it into a python data structure we use `pandas`. This is a preliminary step for any approach and while some libraries may offer custom way to parse data I find that this is just better. Delegating parsing to a second library follows the *single-responsibility* principle. This is at least true for datasets saved in common formats like `csv` or `tsv` or similar. Sometimes we will deal with custom formats like Pytorch's `pt` files: in that case it is obviously better (or sometimes necessary) to take care of data loading with the right library.

In [ ]:
import pandas as pd

We read data from a `csv` file and cast it into a `pandas.DataFrame`

In [ ]:
df = pd.read_csv('./data/house_pricing.csv')
df.head()

This dataset has two feature columns (`sqf` and `rooms`) and a label column (`price`)

Let's assign the features $X$ and the labels $y$ to two different variables

In [ ]:
xy = df.values.T
X = xy[:-1].T
y = xy[-1]

Where the features $X$ are

In [ ]:
X[:5]

and their labels $y$

In [ ]:
y[:5].reshape(-1, 1)

## scikit-learn
Linear regression in `scikit-learn` is as easy as one line of code. To keep this first example as easy as possible, I'm not going to split the data in training and dev sets. I'm just fitting the model to the whole dataset. In a real scenario, there should be a preliminary step of dataset splitting. 

### Single feature
In order for the first example to be as simple as possible and plottable, for now we drop the `rooms` column from the features and we are only left with the `sqf` column. This means that in this first example we are exploring linear dependency between the inhabitable surface and the price of a house.

In [ ]:
X_simple = X[:, 0]
X_simple

Since the `fit()` function that we are using later wants a 2D-vector of shape $(m, n)$  and we only have one feature, we need to reshape the array in the form $(m, 1)$. On the other hand $y$ can either be a 2D or 1D array.

In [ ]:
X_simple = X_simple.reshape(-1, 1)
X_simple[:5]

Building a linear regression model with [scikit-learn](https://scikit-learn.org/) requires the `LinearRegression` class

In [ ]:
from sklearn.linear_model import LinearRegression

Now we can build the model buy instantiating the `LinearRegression` class

In [ ]:
linreg = LinearRegression()

the `linreg` variable contains a linear regression object that allow the computation of the model, but we didn't feed the data to it. Data is fed to the `.fit()` method

In [ ]:
linreg = linreg.fit(X_simple, y)

The parameters and bias of the model are returned with

In [ ]:
linreg.coef_, linreg.intercept_

In [ ]:
fig, ax = plt.subplots()
ax.plot(X_simple, y, ls='none', marker='o')
px = np.linspace(1000, 5000)
ax.plot(px, linreg.predict(px.reshape(-1, 1)))
ax.set_title(f'$\\hat{{y}}={linreg.coef_[0]:.2f} \cdot x '
             f'+ {linreg.intercept_:.2f}$', fontsize=16);

### Multiple Features
We can now introduce the dataset split step that we oversaw in the previous example. In `scikit-learn` splitting the dataset in train and test set is taken care of for us through a function. The proportion of the split can be configured through its arguments.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

We can now fit the all ($n=2$) features of the training set $X^t$

In [ ]:
linreg = LinearRegression().fit(X_train, y_train)

Since this time $X^t \in \mathbb{R}^{m \times 2}$, we have 2 weight parameters and 1 bias parameter

In [ ]:
linreg.coef_, linreg.intercept_

Parameters fitted on the training set can be used to produce prediction from the test set features

In [ ]:
y_pred = linreg.predict(X_test)
y_pred

Predictions can be now compared to the labels of the test set

In [ ]:
from sklearn.metrics import explained_variance_score
explained_variance_score(y_test, y_pred)

## Pytorch
Whereas `scikit-learn` is a high-level library, `Pytorch` is has a much lower-level approach. Many of the things that in `scikit-learn` happen under the hoods, in `Pytorch` need to be done manually.

The steps for training a model in Pytorch as defined in [Pytorch documentation](https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html#sphx-glr-beginner-blitz-neural-networks-tutorial-py) are

1. Load Dataset
2. Make Dataset Iterable
3. Create Model Class
4. Instantiate Model Class
5. Instantiate Loss Class
6. Instantiate Optimizer Class
7. Train Model

The main entry point of the framework is the `torch` module

In [ ]:
import torch

The first noticeable Pytorch feature is that it works using a proprietary data-structure, called a `tensor`. The underlying mathematical concept of tensor is beyond the scope of this article but can be consulted at the [Wikipedia tensor entry](https://en.wikipedia.org/wiki/Tensor). In Pytorch, a `tensor` is (citing the [pytorch tensor documentation](https://pytorch.org/docs/stable/tensors.html)) *a multi-dimensional matrix containing elements of a single data type*.

In [ ]:
X_tensor = torch.tensor(X, dtype=torch.float32)
X_tensor[:5]

As you can notice we had to specify `dtype=np.float32`. This is because the underlying implementation of forward and backward propagation used by Pytorch under the hood would not work with the `int` type.

Furthermore, $y$ tensor would be 1D but this would not comply with requirements of Pytorch methods used below, so we transform it into a column vector with the `unsqueeze(-1)` method. This is equivalent to calling `.reshape(-1, 1)` on a `numpy.array`

In [ ]:
y_tensor = torch.tensor(y, dtype=torch.float32).unsqueeze(-1)
y_tensor[:5]

Since data is in very different scales we need to first normalize it. Here we use [standardization](https://en.wikipedia.org/wiki/Standard_score), which rescales data to have mean $\mu=0$ and standard deviation $\sigma=1$

$$
X_\text{std} = \frac{X - \mu}{\sigma}
$$

In [ ]:
X_tensor_norm = (X_tensor - X_tensor.mean()) / torch.sqrt(X_tensor.var())
X_tensor_norm[:5]

A linear regression model can be built using the `Linear` class from the `nn` module, which initializes bias and weights automatically. Its constructor takes as input the number of columns of the input ($n_X$) and of the output ($n_y$)

In [ ]:
model = torch.nn.Linear(2, 1)

Training the model will require some hyperparameters that we will define in advance for convenience:

* `epochs` is the number of times the model will see all of our training samples;
* `alpha` is the learning rate, which defines how big are the steps takes in updating the parameters;
* `loss_func` is the loss function $\mathcal{L}$ used at training time. In this case we are using the `MSELoss`, which measures the mean squared error (squared L2 norm) between each element in the input $x$ and target $y$;
* `optim` is the optimization algorithm used. In this case we are using `SGD` (Stochastic Gradient Descent). SGD requires us to select the correct $\alpha$. Up to this point we haven't seen that other optimization algorithm exist that automatically adapt $\alpha$ to data (e.g. [ADAM](https://en.wikipedia.org/wiki/Stochastic_gradient_descent#Adam)), so we are sticking with standard SGD.

In [ ]:
epochs = 10
alpha = 0.01
loss_func = torch.nn.MSELoss()
optim = torch.optim.SGD(model.parameters(), lr=alpha)

Now we need to manually run over the epochs and trigger the update of the parameters that will ultimately produce a fitted model

In [ ]:
from torch.autograd import Variable

for epoch in range(epochs):
    inputs = Variable(X_tensor_norm)
    labels = Variable(y_tensor)
    # Clear gradient buffers because from previous epoch
    optim.zero_grad()
    # get output from the model, given the inputs
    outputs = model(inputs)
    # get loss for the predicted output
    loss = loss_func(outputs, labels)
    # get gradients w.r.t to parameters
    loss.backward()
    # update parameters
    optim.step()
    print('epoch {}, loss {}'.format(epoch, loss.item()))

In [ ]:
model(inputs)[:5]

## Pytorch-lightning
Pytorch-lightning is a high-level wrapper for Pytorch that at the same time prevents you from writing much of the boilerplate code needed for a Pytorch model, and automatically adopts the most suited optimization strategies.

It works on top of Pytorch-lightning and scikit-learn (among the others) and really speeds up the design process. For example its additional module `lightning bolts` offers linea regression and logistic regression implementations with `numpy` and `sklearn` bridges for datasets! But their implementations work on multiple GPUs, TPUs and scale dramatically. 

In [ ]:
# from pl_bolts.models.regression import LinearRegression
# import pytorch_lightning as pl
# from pl_bolts.datamodules import SklearnDataModule

# loaders = SklearnDataModule(X_tensor_norm.numpy(), y_tensor.numpy(), num_workers=4)
# model = LinearRegression(input_dim=2, bias=True)
# trainer = pl.Trainer()
# trainer.fit(model, train_dataloader=loaders.train_dataloader(), val_dataloaders=loaders.val_dataloader())
# trainer.test(test_dataloaders=loaders.test_dataloader())